In [1]:
import pandas as pd
import numpy as np
import pyodbc
from datetime import datetime
import datetime as dt
xrange = range
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
import xlsxwriter

#pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.max_columns', 200)

import os
print("The working directory was: {0}".format(os.getcwd()))
os.chdir("C:/hjBoralex/etl/gitcwd")
print("The current working directory is: {0}".format(os.getcwd()))

The working directory was: C:\hjBoralex\etl\gitcwd
The current working directory is: C:\hjBoralex\etl\gitcwd


In [2]:
cwdpath = "C:/Users/hermann.ngayap/Desktop/data/market/markets/in/"

In [16]:
#To import 
prod = pd.read_excel(cwdpath + "template_prod.xlsx")
prod.sort_values(by=['projet'], inplace=True, ignore_index=True)

prod_perc = pd.read_excel(cwdpath + "template_prod.xlsx", sheet_name="prod_perc")
prod_perc = prod_perc.iloc[:,1:]

projet_names_id = pd.read_excel(cwdpath + "template_asset_vmr_planif.xlsx", usecols = ["projet_id", "projet", "en_planif"])
projet_names_id = projet_names_id.loc[projet_names_id["en_planif"] == "Non"]
projet_names_id.sort_values(by=['projet'], inplace=True, ignore_index=True)
projet_names_id.drop("en_planif", axis=1, inplace=True)

In [17]:
projet_names_id.rename(columns={"projet_id":"code", "projet":"projet_names"}, inplace=True)

In [6]:
print(prod.shape)
print(projet_names_id.shape)

(96, 3)
(96, 2)


In [20]:
#To join 2 data frame
frames = [projet_names_id, prod]
df = pd.concat(frames, axis=1, ignore_index=False)

#To create a new column with projet_id
n = 5
df.loc[df['projet'].str[:n] == df['projet_names'].str[:n], 'projet_id'] = df["code"]
df=df[["projet_id", "projet", "p50", "p90"]]

#### TO CHANGE PROD_PERC COLUMN NAMES BY PROJET_ID

In [23]:
n = 5
s = (df.assign(names=df['projet'].str[:n])
    .drop_duplicates('names')
    .set_index('names')['projet_id']
)

prod_id = (pd
   .concat([prod_perc.columns.to_frame().T, prod_perc])
   .rename(columns=lambda x: s.loc[x[:n]])

)

prod_id.reset_index(inplace=True, drop=True)
prod_id = prod_id.iloc[1:,:]

In [25]:
#To export multiple df into one excel file
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(cwdpath + 'template_prod.xlsx', engine='xlsxwriter')
# Write each dataframe to a different worksheet.
prod_id.to_excel(writer, sheet_name="prod_id", float_format="%.5f", index=False)
# Close the Pandas Excel writer and output the Excel file.
writer.save()